In [1]:
import mne
import numpy as np
import os
from glob import glob

from mne_bids import BIDSPath, write_raw_bids

In [8]:
# project directory
proj_dir = os.path.join('/Users/dsj3886/',
                        'data_local',
                        'EAM1',)

# raw data directory (need to specify for your computer)
data_dir = os.path.join(proj_dir, 'EEG_raw')

In [9]:
# output BIDS directory to be created/filled
bids_dir = os.path.join(proj_dir,
                        'data-bids')

In [75]:
# define input EEG file
# TODO: change to loops
sub_label  = '21'
task_label = 'active' # 'passive', 'active'
run_label  = '4'

data_fpath = os.path.join(data_dir,
                          f'sub-{sub_label}_task-{task_label}_run-{run_label}.bdf')

# read in input EEG file
raw_data = mne.io.read_raw_bdf(data_fpath)

Extracting EDF parameters from /Users/dsj3886/data_local/EAM1/EEG_raw/sub-21_task-active_run-4.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...


In [76]:
events_raw = mne.find_events(raw_data, 
                             stim_channel='Status', 
                             initial_event=True)

3603 events found on stim channel Status
Event IDs: [    3     4     7    10   131   132   135   138 65536 65790]


In [77]:
# define event labels and codes (differs per participant)
if int(sub_label) > 19:
    event_dict = {'passive/pos': 1,
                 'passive/neg': 2,
                 'active/pos': 3,
                 'active/neg': 4}
elif int(sub_label) > 13:
    event_dict = {'pos': 1,
                  'neg': 2,}
elif int(sub_label) < 13:
    print('all active coded as 2049.',
         '  need to fix')
    #event_dict = {'pos': 2049,
    #              'neg': 2050,}

In [78]:
# only keep task-related events
events = mne.pick_events(events_raw, include=[1, 2, 3, 4, 2049, 2050])
print('task event codes:', np.unique(events[:,2]))

task event codes: [3 4]


In [79]:
# define the BIDSPath object for converting this EEG file
bids_path = BIDSPath(subject=sub_label, 
                     run=run_label, 
                     task=task_label,
                     datatype='eeg', 
                     root=bids_dir, )

In [80]:
# finally, do the raw-to-BIDS conversion
write_raw_bids(raw_data, 
               bids_path=bids_path,   
               events=events,
               event_id=event_dict,
               overwrite=True
              )

Extracting EDF parameters from /Users/dsj3886/data_local/EAM1/EEG_raw/sub-21_task-active_run-4.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/participants.tsv'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/participants.json'...
Used Annotations descriptions: ['active/neg', 'active/pos']
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_events.tsv'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_events.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/dataset_description.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_eeg.json'...
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/eeg/sub-21_task-active_run-4_channels.tsv'...
Copying data files to sub-21_task-active_run-4_eeg.bdf
Writing '/Users/dsj3886/data_local/EAM1/data-bids/sub-21/sub-21_scans.tsv

BIDSPath(
root: /Users/dsj3886/data_local/EAM1/data-bids
datatype: eeg
basename: sub-21_task-active_run-4_eeg.bdf)